استيراد المكتبات | Import Libraries 

In [11]:
import pandas as pd
import sqlite3 as sql
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules

## Load data from SQLite database

In [ ]:
conn = sql.connect("../data/products.db")
products_df = pd.read_sql_query("SELECT * FROM Products", conn)
invoices_df = pd.read_sql_query("SELECT * FROM Invoices", conn)
conn.close()

,invoice_id,product_id
0,2001,9
1,2001,17
2,2001,1
3,2001,25
4,2002,19
...,...,...
141,2039,11
142,2039,18
143,2040,17
144,2040,10


## Convert invoices to transactions (list of lists of product_ids)

In [18]:
transactions = invoices_df.groupby('invoice_id')['product_id'].apply(list).tolist()

## Create mapping from product_id to product name

In [19]:
id_to_name = products_df.set_index('product_id')['name'].to_dict()

## Convert product_ids to product names

In [20]:
transactions_named = [
    [id_to_name.get(pid, str(pid)) for pid in invoice]
    for invoice in transactions
]


## Save transactions to CSV format

In [21]:
transactions_path = "../data/transactions_named.csv"
with open(transactions_path, 'w', encoding='utf-8') as f:
    for invoice in transactions_named:
        f.write(','.join(invoice) + '\n')

## Convert Data to Data Frame 

In [22]:
te = TransactionEncoder()
te_ary = te.fit(transactions_named).transform(transactions_named)
df_transactions = pd.DataFrame(te_ary, columns=te.columns_)

## Ecxtra Frequent Itemsets

In [23]:
frequent_itemsets = apriori(df_transactions, min_support=0.1, use_colnames=True)

##  Ecxtra Association Rules

In [24]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [25]:
rules[['antecedents', 'consequents', 'support', 'confidence', 'lift']].sort_values(by='lift', ascending=False).head()

,antecedents,consequents,support,confidence,lift
6,(Printer),(Headset),0.125,0.625000,3.571429
7,(Headset),(Printer),0.125,0.714286,3.571429
4,(NAS Storage),(HDMI Cable),0.125,0.454545,1.818182
5,(HDMI Cable),(NAS Storage),0.125,0.500000,1.818182
1,(External HDD),(Conference Webcam),0.100,0.444444,1.777778


In [26]:
rules.to_csv("../data/apriori_rules.csv", index=False)